In [33]:
#Entrainement du modèle à déployer
#1.Importation des librairies
import pandas as pd 
import numpy as np 

import sklearn
### Train Test Split
from sklearn.model_selection import train_test_split
#Gestion unbalanced
import imblearn
from imblearn.over_sampling import SMOTE
#Classifier
import lightgbm
from lightgbm import LGBMClassifier

### Check Accuracy
from sklearn.metrics import accuracy_score

In [34]:
#Importation des données
path = "/Users/adpro/Desktop/Scoring_Model/data_api.csv"
df = pd.read_csv(path)
df = df.set_index("ID")
df.head()

,GENDER,BUSINESS_TYPE,EXT3,REGION_RATING,UNACCOMPANIED,EXT2,INCOME_TYPE,Target
ID,,,,,,,,
100002,0,1,0.139376,2,1,0.262949,1,1
100003,1,0,0.535276,1,0,0.622246,0,0
100004,0,0,0.729567,2,1,0.555912,1,0
100006,1,1,0.535276,2,1,0.650442,1,0
100007,0,0,0.535276,2,1,0.322738,1,0


In [35]:
#pré-proc des données
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

#split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
#resampling SMOTE
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [37]:
#modele
clf = LGBMClassifier(application='binary',
                         is_unbalance=True,
                         metric='auc',
                         max_depth=30,
                         n_estimators=2000,
                         num_leaves=35,
                         subsample=0.1)

In [38]:
#Fiting
clf.fit(X_res,y_res)
#prediction
y_pred=clf.predict(X_test)

In [39]:
score=accuracy_score(y_test,y_pred)
score

0.7206666666666667

In [82]:
##Enregistrement du modèle avec Pickle
import pickle5 as pickle
pickle_out = open("clf.pkl","wb")
pickle.dump(clf, pickle_out)
pickle_out.close()

In [6]:
#test de requete api version fast api: 
# import requests
url = "http://127.0.0.1:8000/predict/{customer_id}"
customer_id = 100002

response = requests.get(url.format(customer_id=customer_id))

if response.status_code == 200:
    result = response.json()
    print(result)
else:
    print("Erreur lors de la requête vers l'API, code de réponse :", response.status_code)
    

Erreur lors de la requête vers l'API, code de réponse : 500


In [ ]:
#test de requete api version flask : 
# import requests
url = "http://127.0.0.1:5000/predict/{customer_id}"
customer_id = 100002

response = requests.get(url.format(customer_id=customer_id))

if response.status_code == 200:
    result = response.json()
    print(result)
else:
    print("Erreur lors de la requête vers l'API, code de réponse :", response.status_code)
    

In [77]:
pickle.format_version

'4.0'